## 아마존 서비스 - S3 설정하기

In [12]:
import os, sys, boto3, dotenv, urllib.parse, requests
from pathlib import Path
dotenv.load_dotenv('../.env')

AWS_ACCESS_KEY = os.getenv("AWS-ACCESS-KEY")
AWS_SECRET_KEY = os.getenv("AWS-SECRET-KEY")
S3_REGION = os.getenv("S3-BUCKET-REGION")
S3_BUCKET_NAME = os.getenv("S3-BUCKET-NAME")

In [13]:
s3_client = boto3.client(
        "s3",
        aws_access_key_id=AWS_ACCESS_KEY,
        aws_secret_access_key=AWS_SECRET_KEY,
        region_name=S3_REGION,
    )

In [20]:
path = Path.cwd()
file_path = path /"../image"/file_name
file_name = "naver.png"

### 업로드

In [21]:
encoded_file_name = urllib.parse.quote(file_name)
s3_client.upload_file(
            file_path, 
            S3_BUCKET_NAME, 
            file_name,
            # ExtraArgs={"ACL": "public-read"}
        )

In [22]:
# S3 URL 생성
file_url = f"https://{S3_BUCKET_NAME}.s3.{S3_REGION}.amazonaws.com/{encoded_file_name}"

In [ ]:
file_url

### 다운로드

In [ ]:
# 파일 다운로드
file_name = 'naver.png'  # S3에 저장된 파일 이름
download_path = './test_1.png'  # 로컬에 저장할 파일 이름

try:
    s3_client.download_file(S3_BUCKET_NAME, file_name, download_path)
    print(f"File downloaded successfully to {download_path}")
except Exception as e:
    print(f"Error occurred while downloading file: {e}")

In [ ]:
response = requests.get(file_url)
if response.status_code == 200:
    with open("./test.png", "wb") as file:
        file.write(response.content)
    print("File downloaded successfully")
else:
    print(f"Failed to download file. HTTP Status Code: {response.status_code}")

### 파일 삭제

In [ ]:
try:
    s3_client.delete_object(Bucket = S3_BUCKET_NAME, Key = file_name)
    print(f"File '{file_name}' successfully deleted from bucket '{S3_BUCKET_NAME}'.")
except Exception as e:
    print(f"Error occurred while deleting file: {e}")